In [9]:
library(scRNAseq)
library(scater)
library(scran)
library(Glimma)
library(edgeR)
library(Seurat)


In [2]:
setwd("/data2st1/sch_backup/data_from_huangyin_128samples/Neuron/data_for_R/HIP")
seo = readRDS("HIP_final_v2.rds")

In [3]:
# scale the nFeature_RNA to 0-1
seo$nFeature_RNAS = (seo$nFeature_RNA - min(seo$nFeature_RNA)) / (max(seo$nFeature_RNA) - min(seo$nFeature_RNA))
# scale the nCount_RNA
seo$nCount_RNAS = (seo$nCount_RNA - min(seo$nCount_RNA)) / (max(seo$nCount_RNA) - min(seo$nCount_RNA))


cell_groups <- unique(seo@meta.data[['celltype.L2']])

In [ ]:
perform_edgeR_analysis <- function(seurat_obj,
                                   group.by, 
                                   compare.by,
                                   batch.by, 
                                   group1, 
                                   group2,
                                   min.cpm = 1,
                                   min.samples = 2,
                                   pval_cutoff = 0.05,
                                   lfc_threshold = 0) {
  
  # Load required libraries
#   if (!requireNamespace("edgeR", quietly = TRUE)) {
#     BiocManager::install("edgeR")
#   }
#   library(edgeR)
  
  # Extract counts and metadata
  counts <- GetAssayData(seurat_obj, layer = "counts")  # Raw counts
  metadata <- seurat_obj@meta.data
  
  # Filter genes (mitochondrial, ribosomal, low-expression)
  filter_genes <- !grepl("^Rp|^mt-", rownames(counts))
  counts <- counts[filter_genes, ]
  
  # Subset cells for comparison
  cells_group1 <- rownames(metadata[metadata[[compare.by]] == group1 & metadata[[group.by]] == "HPF DG GC Glut", ])
  cells_group2 <- rownames(metadata[metadata[[compare.by]] == group2 & metadata[[group.by]] == "HPF DG GC Glut", ])
  counts <- counts[, c(cells_group1, cells_group2)]
  
  # Create DGEList object
  group <- factor(c(rep(group1, length(cells_group1)), rep(group2, length(cells_group2))))
  y <- DGEList(counts = counts, group = group)
  
  # Filter low-expressed genes (CPM-based)
  keep <- rowSums(cpm(y) > min.cpm) >= min.samples
  y <- y[keep, , keep.lib.sizes = FALSE]
  
  # Normalize (TMM)
  y <- calcNormFactors(y)
  
  # Estimate dispersion (for QL F-test)
  #design <- model.matrix(~group)
  batch <- factor(metadata[[batch.by]])
  #design <- model.matrix(~group + batch)  # Add batch as a covariate

  design <- model.matrix(~group + batch, data = metadata)
  y <- estimateDisp(y, design)
  
  # Differential testing (QL F-test recommended for bulk RNA-seq)
  fit <- glmQLFit(y, design)
  qlf <- glmQLFTest(fit, coef = 2)
  
  # Extract results
  res <- topTags(qlf, n = Inf, sort.by = "PValue")$table
  res$gene <- rownames(res)
  res$cell_group <- "HPF DG GC Glut"
  
  # Filter significant DEGs
  res_sig <- res[res$FDR < pval_cutoff & abs(res$logFC) > lfc_threshold, ]
  
  if (nrow(res_sig) == 0) {
    warning("No significant DEGs found at the given thresholds.")
  }
  
  return(list(all_results = res, significant_results = res_sig))
}

In [11]:
seo_M = subset(seo, sex=="M")
seo_F = subset(seo, sex=="F")

# MAST
r.deg_M <- perform_edgeR_analysis (seo_M,
                               group.by = "celltype.L2",
                               compare.by = "treatment",
                               group1 = "control",
                               group2 = "depression",
                               batch.by = "company")


Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 7.6 GiB"


ERROR: Error in perform_edgeR_analysis(seo_M, group.by = "celltype.L2", compare.by = "treatment", : object 'metadata_subset' not found
